In [56]:
import re
import os
import pandas as pd
import emoji
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import contractions

tqdm.pandas()

In [ ]:
data = pd.read_csv("Data/GamesData.csv", usecols=["content", "score", "year", "game_name"])
data["length"] = data["content"].astype(str).str.len()
data = data[data["length"] <= 500]
data = data[data["score"] != 0]
data = data.dropna()
data = data.reset_index(drop=True)
data

C:\Users\Austin\AppData\Local\Temp\ipykernel_15008\1013713984.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(5000, len(x)), random_state=42))


,game_name,content,score,year,length
0,8 Ball Pool,It's very very suppicious that we play versus ...,2,2021,261
1,8 Ball Pool,"I want to say something ,my daily cash reward ...",3,2022,162
2,8 Ball Pool,you have to get through 40 pop ups just to pla...,1,2025,88
3,8 Ball Pool,It seems as if the game is rigged. Looks as if...,2,2020,434
4,8 Ball Pool,Great game/graphics....only thing is the game ...,3,2020,223
...,...,...,...,...,...
99995,Shadow Fight 2,I'm playing this game from many times and it i...,5,2020,51
99996,Shadow Fight 2,Shadow fight 2 is great. You must download thi...,5,2022,53
99997,Shadow Fight 2,this is the best fighting game in my life and ...,5,2025,52
99998,Shadow Fight 2,This game was very exciting.This game mission ...,5,2020,63


In [58]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons 😀😅😂🤣😇🙃🙂
    "\U0001F300-\U0001F5FF"  # symbols & pictographs 🌍🌈⛄
    "\U0001F680-\U0001F6FF"  # transport & map symbols 🚀🚗🛳️
    "\U0001F1E0-\U0001F1FF"  # flags 🇮🇩🇺🇸
    "\U00002702-\U000027B0"  # dingbats ✂️✈️✔️
    "\U000024C2-\U0001F251"  # enclosed characters 🅰️🆗
    "]+",
    flags=re.UNICODE,
)

has_emoji = data["content"].astype(str).apply(lambda x: bool(emoji_pattern.search(x)))

print("Number of reviews containing emoji:", has_emoji.sum())
print("Percentage of reviews with emoji:", round(has_emoji.mean() * 100, 2), "%")

Number of reviews containing emoji: 10828
Percentage of reviews with emoji: 10.83 %


In [59]:
def dedupe_mixed_emoji_bursts(text: str, case_insensitive=True, keep_cap=None):
    def repl(m):
        # all demojized tokens in the matched burst
        toks = re.findall(r':[^\s:]+:', m.group(0))
        seen = set()
        uniq = []
        for t in toks:
            key = t.lower() if case_insensitive else t
            if key not in seen:
                seen.add(key)
                uniq.append(t)
                if keep_cap is not None and len(uniq) >= keep_cap:
                    break
        return ' ' + ' '.join(uniq) + ' '

    # Match any burst of 2+ demojized tokens (allows spaces between)
    # Because we use emoji.demojize(..., delimiters=(' :', ': ')), tokens are spaced.
    return re.sub(r'(?:\s*:[^\s:]+:){2,}', repl, text)


In [60]:
def expand_contractions(text):
    text = contractions.fix(text)
    text = re.sub(r"’", "'", text)  # normalize apostrophes
    return text

StopWords = set(stopwords.words("english"))

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # fallback
    
Lemmatizer = WordNetLemmatizer()

In [61]:
def clean(review):
    #lower text
    review = review.lower()

    #translate emoji
    review = emoji.demojize(review, delimiters=(' :', ': '))
    review = dedupe_mixed_emoji_bursts(review, case_insensitive=True, keep_cap=10)
    review = review.replace("_", " ")
    review = re.sub(r":([a-z0-9 ]+):", r"\1", review)

    #expand contractions
    review = expand_contractions(review)

    #remove non-alphabet and non-numbers 
    review = re.sub(r'[^a-z0-9\s]+', '', review)    
    review = re.sub(r"\s+", " ", review).strip()
    if not review:
        return ""

    #tokenize words
    words = review.split()
    
    #lemmatize words
    tagged_words = pos_tag(words)
    lemmatized = [Lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in tagged_words]

    #remove stopwords
    cleaned = [w for w in lemmatized if w not in StopWords]

    return " ".join(cleaned)

In [62]:
data['content'] = data['content'].progress_apply(clean)
data = data.dropna()
data.to_csv("Data/GamesDataClean.csv", index=False)
data

100%|██████████| 100000/100000 [01:32<00:00, 1083.18it/s]


,game_name,content,score,year,length
0,8 Ball Pool,suppicious play versus cpu cpu always win prov...,2,2021,261
1,8 Ball Pool,want say something daily cash reward video wor...,3,2022,162
2,8 Ball Pool,get 40 pop ups play game good gam3 worth,1,2025,88
3,8 Ball Pool,seem game rig look get 5 game win streak losin...,2,2020,434
4,8 Ball Pool,great gamegraphicsonly thing game fix lose mat...,3,2020,223
...,...,...,...,...,...
99995,Shadow Fight 2,play game many time fun,5,2020,51
99996,Shadow Fight 2,shadow fight 2 great must download game,5,2022,53
99997,Shadow Fight 2,best fighting game life good,5,2025,52
99998,Shadow Fight 2,game excitingthis game mission excited,5,2020,63


In [63]:
data.isna().sum()

game_name    0
content      0
score        0
year         0
length       0
dtype: int64